## <span style="color:Aquamarine">**Hyperparameter Tunning using GridSearchCV and Finding the best model**<span>
- For iris flower dataset in sklearn library, we are going to find out best model and best hyper parameters using GridSearchCV

In [1]:
import pandas as pd
from sklearn.datasets import load_iris
iris=load_iris()

In [2]:
df=pd.DataFrame(iris.data, columns=iris.feature_names)
df.head()

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm)
0,5.1,3.5,1.4,0.2
1,4.9,3.0,1.4,0.2
2,4.7,3.2,1.3,0.2
3,4.6,3.1,1.5,0.2
4,5.0,3.6,1.4,0.2


In [3]:
df['flower']=iris.target
df['flower']=df['flower'].apply(lambda x: iris.target_names[x])
df['target']=iris.target
df[48:55]

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),flower,target
48,5.3,3.7,1.5,0.2,setosa,0
49,5.0,3.3,1.4,0.2,setosa,0
50,7.0,3.2,4.7,1.4,versicolor,1
51,6.4,3.2,4.5,1.5,versicolor,1
52,6.9,3.1,4.9,1.5,versicolor,1
53,5.5,2.3,4.0,1.3,versicolor,1
54,6.5,2.8,4.6,1.5,versicolor,1


### <span style="color:limegreen">**Approach 1: Use train_test_split and manually tune parameters by trial and error**</span>

In [4]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test=train_test_split(iris.data, iris.target, test_size=0.3, random_state=42)

In [5]:
from sklearn.svm import SVC
model=SVC(C=10, kernel='poly', gamma='auto')
model.fit(X_train, y_train)
model.score(X_test, y_test)

1.0

### <span style="color:limegreen">**Approach 2: Use K Fold Cross validation**</span>

**Manually try suppling models with different parameters to cross_val_score function with 5 fold cross validation**

In [11]:
from sklearn.model_selection import cross_val_score
import numpy as np


In [14]:
scores1=cross_val_score(SVC(C=5, kernel='poly', gamma='auto'), iris.data, iris.target, cv=5)
scores1

array([1.        , 1.        , 0.9       , 0.93333333, 1.        ])

In [15]:
np.average(scores1)

0.9666666666666666

In [8]:
cross_val_score(SVC(kernel='linear',C=10,gamma='auto'),iris.data, iris.target, cv=5)

array([1.        , 1.        , 0.9       , 0.96666667, 1.        ])

In [9]:
cross_val_score(SVC(kernel='rbf', C=20, gamma='auto'), iris.data, iris.target, cv=5)

array([0.96666667, 1.        , 0.9       , 0.96666667, 1.        ])

**We Can use for loop to avoid manually tunning the parameters**

In [18]:
kernels=['linear', 'poly', 'rbf']
C=[1,10,20]

avg_scores={}

for k in kernels:
    for c in C:
        cv_scores=cross_val_score(SVC(kernel=k, C=c, gamma='auto'), iris.data, iris.target, cv=5)
        avg_scores[k+'_'+str(c)]=np.average(cv_scores)
avg_scores

{'linear_1': 0.9800000000000001,
 'linear_10': 0.9733333333333334,
 'linear_20': 0.9666666666666666,
 'poly_1': 0.9666666666666666,
 'poly_10': 0.9666666666666666,
 'poly_20': 0.9533333333333334,
 'rbf_1': 0.9800000000000001,
 'rbf_10': 0.9800000000000001,
 'rbf_20': 0.9666666666666668}

### **From the above we can conclude that linear with C=1 will give the best performance**

### <span style="color:limegreen">**Approach 3: Use GridSearchCV**</span>
**GridSearchCV does exactly same thing as for loop above but in a more efficient way**

In [20]:
from sklearn.model_selection import GridSearchCV
clf=GridSearchCV(SVC(gamma='auto'),{
    'C':[1,10,20],
    'kernel':['linear', 'poly', 'rbf']
}, cv=5, return_train_score=False)

clf.fit(iris.data, iris.target)
clf.cv_results_

{'mean_fit_time': array([0.00080328, 0.00198312, 0.00086193, 0.00040035, 0.00072908,
        0.00272245, 0.        , 0.00624142, 0.        ]),
 'std_fit_time': array([0.00040176, 0.00123324, 0.00044469, 0.00049033, 0.0010457 ,
        0.00446176, 0.        , 0.00634355, 0.        ]),
 'mean_score_time': array([0.        , 0.00034552, 0.00036459, 0.00039997, 0.00020623,
        0.00033317, 0.        , 0.00013604, 0.        ]),
 'std_score_time': array([0.        , 0.00043172, 0.00045006, 0.00048986, 0.00041246,
        0.00042161, 0.        , 0.00027208, 0.        ]),
 'param_C': masked_array(data=[1, 1, 1, 10, 10, 10, 20, 20, 20],
              mask=[False, False, False, False, False, False, False, False,
                    False],
        fill_value='?',
             dtype=object),
 'param_kernel': masked_array(data=['linear', 'poly', 'rbf', 'linear', 'poly', 'rbf',
                    'linear', 'poly', 'rbf'],
              mask=[False, False, False, False, False, False, False, Fals

In [23]:
df=pd.DataFrame(clf.cv_results_)
df

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_C,param_kernel,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.000803,0.000402,0.000000,0.000000,1,linear,"{'C': 1, 'kernel': 'linear'}",0.966667,1.000000,0.966667,0.966667,1.0,0.980000,0.016330,1
1,0.001983,0.001233,0.000346,0.000432,1,poly,"{'C': 1, 'kernel': 'poly'}",1.000000,1.000000,0.900000,0.933333,1.0,0.966667,0.042164,6
2,0.000862,0.000445,0.000365,0.000450,1,rbf,"{'C': 1, 'kernel': 'rbf'}",0.966667,1.000000,0.966667,0.966667,1.0,0.980000,0.016330,1
3,0.000400,0.000490,0.000400,0.000490,10,linear,"{'C': 10, 'kernel': 'linear'}",1.000000,1.000000,0.900000,0.966667,1.0,0.973333,0.038873,4
4,0.000729,0.001046,0.000206,0.000412,10,poly,"{'C': 10, 'kernel': 'poly'}",1.000000,1.000000,0.900000,0.933333,1.0,0.966667,0.042164,6
5,0.002722,0.004462,0.000333,0.000422,10,rbf,"{'C': 10, 'kernel': 'rbf'}",0.966667,1.000000,0.966667,0.966667,1.0,0.980000,0.016330,1
6,0.000000,0.000000,0.000000,0.000000,20,linear,"{'C': 20, 'kernel': 'linear'}",1.000000,1.000000,0.900000,0.933333,1.0,0.966667,0.042164,6
7,0.006241,0.006344,0.000136,0.000272,20,poly,"{'C': 20, 'kernel': 'poly'}",0.966667,0.966667,0.900000,0.933333,1.0,0.953333,0.033993,9
8,0.000000,0.000000,0.000000,0.000000,20,rbf,"{'C': 20, 'kernel': 'rbf'}",0.966667,1.000000,0.900000,0.966667,1.0,0.966667,0.036515,5


In [25]:
df[['param_C', 'param_kernel', 'mean_test_score']]

,param_C,param_kernel,mean_test_score
0,1,linear,0.980000
1,1,poly,0.966667
2,1,rbf,0.980000
3,10,linear,0.973333
4,10,poly,0.966667
5,10,rbf,0.980000
6,20,linear,0.966667
7,20,poly,0.953333
8,20,rbf,0.966667


In [26]:
clf.best_params_

{'C': 1, 'kernel': 'linear'}

In [27]:
clf.best_score_

0.9800000000000001

In [28]:
clf.best_estimator_

SVC(C=1, gamma='auto', kernel='linear')

In [29]:
dir(clf)

['__abstractmethods__',
 '__annotations__',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__setstate__',
 '__sizeof__',
 '__sklearn_clone__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_abc_impl',
 '_build_request_for_signature',
 '_check_feature_names',
 '_check_n_features',
 '_check_refit_for_multimetric',
 '_estimator_type',
 '_format_results',
 '_get_default_requests',
 '_get_metadata_request',
 '_get_param_names',
 '_get_tags',
 '_more_tags',
 '_parameter_constraints',
 '_repr_html_',
 '_repr_html_inner',
 '_repr_mimebundle_',
 '_required_parameters',
 '_run_search',
 '_select_best_index',
 '_validate_data',
 '_validate_params',
 'best_estimator_',
 'best_index_',
 'best_params_',
 'best_score_',
 '

**For large datasets, we can use RandomizedSearchCV to reduce the number of iterations and random combination of parameters. This is helpful when we have too many parameters to try. It also reduces the number of permutations and combinations required**

In [33]:
from sklearn.model_selection import RandomizedSearchCV
rscv=RandomizedSearchCV(SVC(gamma='auto'), {
    'C': [1,10,20],
    'kernel': ['linear','poly','rbf']
},cv=5, return_train_score=False, n_iter=2)
rscv.fit(iris.data, iris.target)
# rscv.cv_results_

pd.DataFrame(rscv.cv_results_)[['param_C', 'param_kernel', 'mean_test_score']]



,param_C,param_kernel,mean_test_score
0,20,poly,0.953333
1,1,rbf,0.980000


In [34]:
rscv.best_score_

0.9800000000000001

In [35]:
rscv.best_estimator_

SVC(C=1, gamma='auto')

In [36]:
rscv.best_params_

{'kernel': 'rbf', 'C': 1}

### **Using different models with different hyperparameters**

In [38]:
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression

In [42]:
model_params={
    'svm':{
        'model': SVC(gamma='auto'),
        'params': {
            'C': [1,10,20],
            'kernel': ['rbf', 'poly', 'linear']
        }
    },
    'random_forest': {
        'model': RandomForestClassifier(),
        'params': {
            'n_estimators': [10, 50, 100],
        }
    },
    'logistic_regression': {
        'model': LogisticRegression(solver='liblinear', multi_class='auto'),
        'params': {
            "C": [1,5,10 ]
        }
    }
}

In [43]:
scores=[]

for model_name, mparam in model_params.items():
    clf=GridSearchCV(mparam['model'], mparam['params'], cv=5, return_train_score=False)
    clf.fit(iris.data, iris.target)
    scores.append({
        'model': model_name,
        'best_score': clf.best_score_,
        'best_params': clf.best_params_
    })


df=pd.DataFrame(scores, columns=['model', 'best_score', 'best_params'])
df

,model,best_score,best_params
0,svm,0.980000,"{'C': 1, 'kernel': 'rbf'}"
1,random_forest,0.966667,{'n_estimators': 10}
2,logistic_regression,0.966667,{'C': 5}
